# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
import pandas as pd
import numpy as np
import glob
pd.options.mode.chained_assignment = None

DATA_FOLDER = './Data/' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

### GUINEA

First of all, we concatenate all the files in the guinea folder and we only take the 3 columns that interests us : the date, the description and the total for the whole country. We also parse the date with the right format.
Then we put the date as the index.

In [ ]:
path_guinea = DATA_FOLDER+"ebola/guinea_data/"
files_guinea = glob.glob(path_guinea+"*.csv")

df_guinea= pd.concat((pd.read_csv(f,usecols=["Date","Description","Totals"],parse_dates=["Date"]) for f in files_guinea),ignore_index=True)
df_guinea.index = df_guinea.Date
df_guinea = df_guinea.drop('Date',axis=1)

Now we filter the only 2 rows we need, here they are the new cases and the new deaths. We notice the first file uses a different description for the new deaths, so we replace it with the one that is used in the other files.

In [ ]:
filtered_guinea = df_guinea[(df_guinea.Description == "Total new cases registered so far") | (df_guinea.Description == "New deaths registered") | (df_guinea.Description == "New deaths registered today")]
filtered_guinea = filtered_guinea.replace("New deaths registered today","New deaths registered")

The pivot is just a convenient way of looking at it, we don't actually change the structure of the dataframe, because the current structure makes the following easier.

In [ ]:
filtered_guinea.pivot(columns="Description",values="Totals")

We now extract one serie for each (cases and deaths) and convert the values into integers.

In [ ]:
cases_guinea = filtered_guinea[filtered_guinea.Description == "Total new cases registered so far"].Totals.astype(str).astype(int)
deaths_guinea = filtered_guinea[filtered_guinea.Description == "New deaths registered"].Totals.astype(str).astype(int)
cases_guinea

This allows us to average by month quite easily :

In [ ]:
avg_cases_guinea = cases_guinea.resample('M').mean()
avg_deaths_guinea = deaths_guinea.resample('M').mean()
avg_cases_guinea

And we can finally present the data in a nice, concise way.

In [ ]:
avg_guinea = pd.DataFrame({"avg new cases":avg_cases_guinea.values,"avg new deaths":avg_deaths_guinea.values},index=avg_cases_guinea.index.strftime('%B'))
avg_guinea

### LIBERIA

We proceed the same way, only with different names for columns and rows.

In [ ]:
path_liberia = DATA_FOLDER+"ebola/liberia_data/"
files_liberia = glob.glob(path_liberia+"*.csv")

df_liberia= pd.concat((pd.read_csv(f,usecols=["Date","Variable","National"],parse_dates=["Date"]) for f in files_liberia),ignore_index=True)
df_liberia.columns=["Date","Description","Totals"] #for clarity we use the same column names as before
df_liberia.index = df_liberia.Date
df_liberia = df_liberia.drop('Date',axis=1)

In [ ]:
filtered_liberia = df_liberia[(df_liberia.Description.str.contains("New Case|New case")) | (df_liberia.Description == "Newly reported deaths")]

Here, we have to considerate all the different types of new cases and sum them by day.

In [ ]:
separate_cases = filtered_liberia[filtered_liberia.Description != "Newly reported deaths"]
cases_liberia = separate_cases.groupby(separate_cases.index).sum().Totals
deaths_liberia = filtered_liberia[filtered_liberia.Description == "Newly reported deaths"].Totals

In [ ]:
avg_cases_liberia = cases_liberia.resample('M').mean()
avg_deaths_liberia = deaths_liberia.resample('M').mean()

In [ ]:
avg_liberia = pd.DataFrame({"avg new cases":avg_cases_liberia.values,"avg new deaths":avg_deaths_liberia.values},index=avg_cases_liberia.index.strftime('%B'))
avg_liberia

### SIERRA LEONE

In [ ]:
path_sierra = DATA_FOLDER+"ebola/sl_data/"
files_sierra = glob.glob(path_sierra+"*.csv")

df_sierra= pd.concat((pd.read_csv(f,usecols=["date","variable","National"],parse_dates=["date"]) for f in files_sierra),ignore_index=True)
df_sierra.columns=["Date","Description","Totals"] #for clarity we use the same column names as before
df_sierra.index = df_sierra.Date
df_sierra = df_sierra.drop('Date',axis=1)

In [ ]:
filtered_sierra = df_sierra[(df_sierra.Description.str.contains("new_suspected|new_probable|new_confirmed")) | (df_sierra.Description == "etc_new_deaths")]

As for Liberia, we have to sum all the different types of new cases (suspected, probable, confirmed).

In [ ]:
separate_cases = filtered_sierra[filtered_sierra.Description != "etc_new_deaths"].Totals.astype(str).astype(float)
cases_sierra = separate_cases.groupby(separate_cases.index).sum()
deaths_sierra = filtered_sierra[filtered_sierra.Description == "etc_new_deaths"].Totals.astype(str).astype(float)

In [ ]:
avg_cases_sierra = cases_sierra.resample('M').mean()
avg_deaths_sierra = deaths_sierra.resample('M').mean()

In [ ]:
avg_sierra = pd.DataFrame({"avg new cases":avg_cases_sierra.values,"avg new deaths":avg_deaths_sierra.values},index=avg_cases_sierra.index.strftime('%B'))
avg_sierra

### Concatenating all the results in one dataframe

We can note that one value, the average number of new cases in Liberia in december, is strongly out of the domain formed by the other values. This is due to the data (probably wrong) and not the calculations.

In [ ]:
final_result = pd.concat([avg_guinea,avg_liberia,avg_sierra],axis=1,keys=["Guinea","Liberia","Sierra"]).reindex(avg_liberia.index)
final_result.fillna("unknown",inplace=True)
final_result

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Read MID files

path=DATA_FOLDER+"microbiome/"
MID_files = glob.glob(path+"MID*.xls")
MID_files

In [ ]:
df_microbiome=pd.concat((pd.read_excel(f,index_col=0,header=None) for f in MID_files),axis=1)
df_microbiome.columns=['MID1','MID2','MID3','MID4','MID5','MID6','MID7','MID8','MID9']
df_microbiome=df_microbiome.transpose()
df_microbiome.index.names=['BARCODE']
df_microbiome

In [ ]:
df_microbiome.columns=[i for i in range(0,9)]

In [ ]:
df_metadata=pd.read_excel('Data/microbiome/metadata.xls',index_col='BARCODE')
df_metadata

In [ ]:
df_merged=pd.merge(df_microbiome, df_metadata, left_index=True, right_index=True)
df_merged

In [ ]:
df_merged.fillna('unknown',inplace=True)
df_merged

In [ ]:
df_merged.index.is_unique

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here